## This notebook will guide you through the use of the chronix2grid API. You'll be able to separately generate loads, renewable productions and the corresponding dispatch of the other generators.

In [1]:
import os
import sys

import cufflinks as cf

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline
print("This notebook uses the last version of grid2op. You can install it with:\n"\
      "\t{} -m pip install grid2op".format(sys.executable))
import grid2op
if grid2op.__version__ < "0.6.0":
    raise RuntimeError("Impossible to run this notebook without grid2op version 0.6.0 installed.")
from grid2op.Chronics import ChangeNothing
from grid2op.Plot import PlotMatplotlib

import chronix2grid.generation.generate_chronics as gen
import chronix2grid.generation.generation_utils as gu
import chronix2grid.generation.consumption.generate_load as gen_loads
import chronix2grid.generation.renewable.generate_solar_wind as gen_enr
import chronix2grid.generation.dispatch.generate_dispatch as gen_dispatch
import chronix2grid.generation.dispatch.utils as du
import chronix2grid.generation.dispatch.EconomicDispatch as ec
from chronix2grid.generation.dispatch.EDispatch_L2RPN2020 import run_economic_dispatch
import chronix2grid.kpi.main as kpis
from chronix2grid.main import create_directory_tree
import chronix2grid.constants as cst

This notebook uses the last version of grid2op. You can install it with:
	/home/vrenault/Projects/ChroniX2Grid/venv_test/bin/python -m pip install grid2op


/home/vrenault/Projects/ChroniX2Grid/venv_test/lib/python3.6/site-packages/grid2op/Plot/EpisodeReplay.py:34: UserWarning:

The final video will not be saved as "imageio" and "imageio_ffmpeg" packages cannot be imported. Please try "/home/vrenault/Projects/ChroniX2Grid/venv_test/bin/python -m pip install imageio imageio-ffmpeg"



In [2]:
compute_kpis = True  # The computation of KPIs can take some time...
seed_reproducible = True  # Make sure to use the same seeds to reproduce results

# Parameters

## General parameters

In [5]:
### CONSTANT

root_dir = %pwd

# define your input folder
INPUT_FOLDER = os.path.join(root_dir, os.pardir, 'input_data')
OUPTUT_FOLDER = os.path.join(root_dir, os.pardir, 'output')
# Detailed configuration to set in <INPUT_FOLDER>/<CASE>/params.json
start_date = "2012-07-01"
weeks = 4
n_scenarios = 2

## Generation step of chronix2grid

CASE = 'case118_l2rpn_wcci'
path_case = os.path.join(INPUT_FOLDER, 'generation', CASE)
grid_path = os.path.join(path_case, "L2RPN_2020_case118_redesigned.json")

generation_output_folder, kpi_output_folder = create_directory_tree(
    CASE, start_date, OUPTUT_FOLDER, cst.SCENARIO_FOLDER_BASE_NAME, n_scenarios, 
    'LRTK', warn_user=False)

# Load the environment

In [6]:
env118_withoutchron = grid2op.make(
    "blank",  # to generate a blank environment
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=ChangeNothing, # tell it to change nothing (not the most usable environment...)
)

# Check the Energy Mix apriori

In [9]:
%run ../chronix2grid/kpi/Generator_parameter_checker.py
Target_EM_percentage=pd.DataFrame(data=[4,6,35,15,40],columns=['target_energy_mix'],
                                  index=['solar','wind','nuclear','hydro','thermal'])

PeakLoad = 4200
AverageLoad = 2800
    
CapacityFactor=pd.DataFrame(data=[15,25,95,30,np.nan],columns=['capacity_factor'],
                            index=['solar','wind','nuclear','hydro','thermal'])
Capacity_df=EnergyMix_AprioriChecker(env118_withoutchron,Target_EM_percentage, PeakLoad, AverageLoad, CapacityFactor )
Capacity_df

,target_energy_mix,pmax,capacity_mix,capacity_factor,Apriori_energy_mix,revised_pmax
solar,4,746.4,10.0,15.0,3.998571,746.666667
wind,6,672.0,9.0,25.0,6.000000,672.000000
nuclear,35,1200.0,16.1,95.0,40.714286,1031.578947
hydro,15,1750.0,23.4,30.0,18.750000,1400.000000
thermal,40,3100.0,41.5,NaN,30.537143,3168.421053


# II Generate the data

**You can set generation configuration such as number of scenarios, start date, number of weeks, noise intensities, timestep... in INPUT_FOLDER/CASE/params.json**

In [ ]:
if seed_reproducible:
    seeds = [181791698]  # for reproducibility - otherwise comment here and uncomment below
else:
    seeds = [np.random.randint(low=0, high=2**31) for _ in range(n_scenarios)]

In [ ]:
## Reading parameters
year, params, loads_charac, prods_charac, load_weekly_pattern, solar_pattern, params_opf_auto = \
    gu.read_all_configurations(weeks, start_date, CASE, os.path.join(INPUT_FOLDER, 'generation'), 
                               generation_output_folder)
print(year)

scen_name_generator = gu.folder_name_pattern(cst.SCENARIO_FOLDER_BASE_NAME, n_scenarios)

# II A) Generate loads and renewables

In [ ]:
# Separate generation for load and renewables

# Launch load generation
for i, seed in enumerate(seeds):
    scenario_name = scen_name_generator(i)
    scenario_folder_path = os.path.join(generation_output_folder, scenario_name)
    print("================ Generating scenario number "+str(i)+" ================")
    load, load_forecasted = gen_loads.main(scenario_folder_path, seed, params, loads_charac, 
                                           load_weekly_pattern, write_results = True)

    print('\n')

## Check load hypothesis (peak and average)

**if this differs by too much, you should update the computation of the Energy Mix a priori and revise some calibration if not satisfactory**

In [ ]:
CurrentPeakLoad = load.sum(axis=1).max()
print('the expected peak load was: ' + str(PeakLoad))
print('the actual peak load is: ' + str(CurrentPeakLoad))

In [ ]:
CurrentAverageLoad = load.sum(axis=1).mean()
print('the expected average load was: ' + str(AverageLoad))
print('the actual average load is: ' + str(CurrentAverageLoad))

## Generate Renewables

In [ ]:
# Launch solar and wind generation
for i, seed in enumerate(seeds):
    print("================ Generating scenario number "+str(i)+" ================")
    
    scenario_name = scen_name_generator(i)
    scenario_folder_path = os.path.join(generation_output_folder, scenario_name)
    
    prod_solar, prod_solar_forecasted, prod_wind, prod_wind_forecasted = gen_enr.main(
        scenario_folder_path, seed, params, prods_charac, solar_pattern, write_results = True)
    print('\n')

In [ ]:
plt.plot(solar_pattern)

## Check Ramps and Pmin/Pmax Generator parameters A priori

### Select the scenario you want to check first

In [ ]:
generation_output_folder

In [ ]:
%run ../chronix2grid/kpi/Generator_parameter_checker.py

losses_pct = params_opf_auto["losses_pct"]  # losses as pct of load
[isThermalInTrouble, isNuclearInTrouble, IsRampUpInTrouble, IsRampDownInTrouble] = Ramps_Pmax_Pmin_APrioriCheckers(
    env118_withoutchron, Capacity_df, generation_output_folder, losses_pct, PeakLoad)

In [ ]:
print("Are the thermal reactors \"in trouble\": {}".format(isThermalInTrouble))
print("Are the nuclear reactors \"in trouble\": {}".format(isNuclearInTrouble))
print("Are the ramp up \"in trouble\": {}".format(IsRampUpInTrouble))
print("Are the ramp down \"in trouble\": {}".format(IsRampDownInTrouble))

In [ ]:
%run ../chronix2grid/kpi/Generator_parameter_checker.py
Aposteriori_renewableCapacityFactor_Checkers(env118_withoutchron, Capacity_df, generation_output_folder)

## Compute some KPIs for solar, wind and load only

#### Benchmark "France" is set as reference in INPUT_FOLDER/kpi/paramsKPI.json
Images are saved in OUTPUT_FOLDER/kpi/CASE/start_date/SCENARIO/images

In [ ]:
# If you just want to save a lot of plots without showing it, uncomment this line.
# If you want to compute more than 1 scenario, it is recommended not to show the plots on notebook
#%%capture
if compute_kpis:
    # Chose number of scenarios to compute KPIs (it can be long to compute it for a lot of scenarios)
    n_scenarios_kpis = 1

    # Computation
    wind_solar_only = True
    scenario_names = gu.folder_name_pattern(cst.SCENARIO_FOLDER_BASE_NAME, n_scenarios_kpis)
    kpis.main(os.path.join(INPUT_FOLDER, cst.KPI_FOLDER_NAME), generation_output_folder,
              scenario_names, kpi_output_folder, year, CASE,
              n_scenarios_kpis, wind_solar_only, params, loads_charac, prods_charac, scenario_id=0)

## II B) Run an economic dispatch 


You need to install the solver that pypsa is calling. For instance cbc solver. On Fedora do `dnf install coin-or-Cbc.x86_64`

Create The EconomicDispatch instance : a high level wrapper around a Pypsa net

In [ ]:
dispatcher = ec.Dispatcher.from_gri2op_env(env118_withoutchron)
dispatcher.modify_marginal_costs({'hydro': 36})
dispatcher.read_hydro_guide_curves(os.path.join(INPUT_FOLDER, 'generation/patterns', 'hydro_french.csv'))

In [ ]:
dispatcher.plot_ramps()

## Dispatch Parameters 

In [ ]:
# Overwrite the params_opf_auto dictionary

losses_pct = 1.0
DispatchByCarrierOnly=False

params_opf = {
    'step_opf_min': 5,
    'mode_opf': 'month',
    'reactive_comp': 1.0,
    'losses_pct': losses_pct,
    'dispatch_by_carrier': DispatchByCarrierOnly,
    'pyomo': False,
    'solver_name': 'cbc'
}

### Run opf

In [ ]:
for scenario_name in os.listdir(generation_output_folder):

    if scenario_name in ['.DS_Store']:
        continue

    scenario_folder_path = os.path.join(generation_output_folder, scenario_name)
    print(scenario_folder_path)
    dispatcher.read_load_and_res_scenario(os.path.join(scenario_folder_path, 'load_p.csv.bz2'),
                                        os.path.join(scenario_folder_path, 'prod_p.csv.bz2'),
                                        scenario_name=scenario_name)
    hydro_constraints = dispatcher.make_hydro_constraints_from_res_load_scenario()
    agg_load_without_renew = dispatcher.net_load(losses_pct, name=dispatcher.loads.index[0])

    # Example of how to extract info on the largest ramps
    print(f'5 largest ramps reached by the agg_load_without_renew:')
    print(dispatcher.nlargest_ramps(5, losses_pct))

    # Run Economic Disptach using submodule EDisptach_L2RPN_2020
    # **  **  **  **  **  **  **  **  **  **  **  **  **  **
    dispatch_results = dispatcher.run(
        agg_load_without_renew,
        params=params_opf,
        gen_constraints=hydro_constraints,
        ramp_mode=run_economic_dispatch.RampMode.hard,
        by_carrier=DispatchByCarrierOnly,  # True to run the dispatch only aggregated generators by carrier,
        pyomo=False,
        solver_name='cbc'
    )

    chronix_scenario = dispatch_results.chronix

    # save prods chronics
    dispatcher.save_results(params, scenario_folder_path)

# TODO if there are failures, write it somewhere, for now it's only detected in the very verbose output cell.
# for example you can do a report at the end 'looking like failures for scenariis xxx'

chronix_scenario is an object containing all the time series related to the studied scenario : 
- chronix_scenario.name gives the name of the scenario
- chronix_scenario.wind_p (resp. solar_p, prods_dispatch, loads, marginal_prices) gives the Wind DataFrame (resp. Solar, Dispatched generators, loads, marginal_prices)

This object should be manipulated in the sequel.

### The code below (up until the kpis) will  not run if  run_automated_dispatch = True

In [ ]:
#check that the max net load is similar than after generating loads and renewables
agg_load_without_renew.max()

In [ ]:
#look at the slcack bus generation for case 118
#dispatch_results.chronix.prods_dispatch['gen_68_37'].iplot(kind='scatter', filename='cufflinks/cf-simple-line')

### Check That Pypsa does not violate the ramps

In [ ]:
StatsRamps=dispatch_results.chronix.prods_dispatch.diff().describe()
maxRamps=StatsRamps.loc['max']
maxRamps[maxRamps>=10]

In [ ]:
gen_names=list(env118_withoutchron.name_gen)
matchIDs=[gen_names.index(el) for el in list(StatsRamps)]
gen_ramps=env118_withoutchron.gen_max_ramp_up[matchIDs]
gen_subIds=env118_withoutchron.gen_to_subid[matchIDs]

RampsToHigh=((maxRamps>gen_ramps).values & (gen_ramps!=0))
print('\n generation above their max rated ramps')
print(gen_subIds[RampsToHigh])
print('\n max ramps in environement for generation above their max rated ramps')
print(pd.DataFrame(gen_ramps,index=list(StatsRamps)).loc[RampsToHigh])
print('\n max ramps after pypsa')
print(maxRamps[RampsToHigh])

check that very high ramps happen when switching months

In [ ]:
Ramps=dispatch_results.chronix.prods_dispatch.diff()
Ramps.iplot(kind='scatter', filename='cufflinks/cf-simple-line')

### Plot the dispatch

In [ ]:
# Concatenate renewable dispatch

#Becareful:check years of opf_dispatch and dispatch
print(chronix_scenario.wind_p.index[0])
print(chronix_scenario.prods_dispatch.index[0])

In [ ]:
if(chronix_scenario.wind_p.index[0] != chronix_scenario.prods_dispatch.index[0]):
    chronix_scenario.prods_dispatch.index=chronix_scenario.wind_p.index
if DispatchByCarrierOnly:
    chronix_scenario.prods_dispatch=chronix_scenario.prods_dispatch[['nuclear','hydro','thermal']]#makesure nuclear comesfirst, for good plotting after

full_opf_dispatch = pd.concat(
    [chronix_scenario.prods_dispatch, chronix_scenario.wind_p, chronix_scenario.solar_p],
    axis=1
)

# Keep same order as grid2op
if not DispatchByCarrierOnly:
    full_opf_dispatch = full_opf_dispatch[env118_withoutchron.name_gen].round(2)

In [ ]:
if not DispatchByCarrierOnly:
    nuclear_names = dispatcher.generators[dispatcher.generators.carrier == 'nuclear'].index
    hydro_names = dispatcher.generators[dispatcher.generators.carrier == 'hydro'].index
    thermal_names = dispatcher.generators[dispatcher.generators.carrier == 'thermal'].index

    dispatch_by_fleet=pd.concat([ dispatcher.wind_p, dispatcher.solar_p], axis=1)
    dispatch_by_fleet['nuclear'] = full_opf_dispatch[nuclear_names].sum(axis=1).to_frame('Nuclear')
    dispatch_by_fleet['hydro'] = full_opf_dispatch[hydro_names].sum(axis=1)
    dispatch_by_fleet['thermal'] = full_opf_dispatch[thermal_names].sum(axis=1)
    #dispatch_by_fleet=pd.concat([dispatch_by_fleet, dispatch.wind_p, dispatch.solar_p], axis=1)

    dispatch_by_fleet.loc[dispatch_by_fleet['thermal'] < 0, 'thermal'] = 0

    # grid2op env starts in 2007 but read loads are in 2012...
    #dispatch_by_fleet = dispatch_by_fleet.loc[dispatch_by_fleet.index.year == 2007,:]

    dispatch_by_fleet.plot(figsize=(20, 8), title='Dispatch over 1 year', kind='area')

else:
    dispatch_by_fleet=full_opf_dispatch
    
    dispatch_by_fleet.loc[dispatch_by_fleet['thermal'] < 0, 'thermal'] = 0 #due to numeric approximation,some thermal values  could be negative
    dispatch_by_fleet.loc[dispatch_by_fleet['hydro'] < 0, 'hydro'] = 0
    #full_opf_dispatch[full_opf_dispatch['thermal']<0]['thermal'].hist()
    dispatch_by_fleet.plot(figsize=(20, 8), title='Dispatch over 1 year', kind='area')


In [ ]:
dispatch_by_fleet[['nuclear','hydro','thermal']].plot(figsize=(20, 8), title='Dispatch over 1 year - no renewable', kind='area')

In [ ]:
WeekNumber=24
dispatch_by_fleet.iloc[(288*7*WeekNumber):(288*7*(WeekNumber+1)), :].plot(figsize=(20, 8), title='Dispatch over 1 week', kind='area')

In [ ]:
dispatch_by_fleet[['nuclear','hydro','thermal']].iloc[(288*7*WeekNumber):(288*7*(WeekNumber+1)), :].plot(figsize=(20, 8), title='Dispatch over 1 week - no renewable', kind='area')

Check Hydro 

In [ ]:
#In june, Hydro might be high and the minimum hydro production to respect forces nuclear to decrease its production
if not(dispatch_by_fleet[['hydro']].sum().values==0):
    minHydroPattern=dispatcher._min_hydro_pu
    nCols=minHydroPattern.shape[1]
    minHydroPattern.iloc[:,0].plot()

In [ ]:
#In june, Hydro might be high and the minimum hydro production to respect forces nuclear to decrease its production
if not(dispatch_by_fleet[['hydro']].sum().values==0):
    maxHydroPattern=dispatcher._max_hydro_pu
    nCols=maxHydroPattern.shape[1]
    maxHydroPattern.iloc[:,0].plot()

In [ ]:
# Validate whether they have same order
np.all(full_opf_dispatch.columns == env118_withoutchron.name_gen)

Check Energy Mix of Dispatch and capacity factors

In [ ]:
CurrentAverageLoad=load.sum(axis=1).mean()
dispatch_by_fleet[['nuclear','hydro','thermal']].mean()/CurrentAverageLoad

In [ ]:
dispatch_by_fleet[['nuclear','hydro','thermal']].mean()/dispatch_by_fleet[['nuclear','hydro','thermal']].max()

#### You have to set "eco2mix" as comparison in INPUT_FOLDER/kpi/paramsKPI.json
**Images were not designed to be plot on a notebook but to be saved as png or zoomable in IMAGES_FOLDER**. In particular, yearly productions and energy mix are better to watch in their written files

In [ ]:
%%capture
if compute_kpis:
    wind_solar_only = False
    n_scenarios_kpis = 1
    scenario_names = gu.folder_name_pattern(cst.SCENARIO_FOLDER_BASE_NAME, n_scenarios_kpis)
    kpis.main(os.path.join(INPUT_FOLDER, cst.KPI_FOLDER_NAME), generation_output_folder, 
              scenario_names, kpi_output_folder, year, CASE,
              n_scenarios, wind_solar_only, params, loads_charac, prods_charac, scenario_id=0)

Html Link to jump here
<a id='load_generated_chronics'></a>

# III Create an environment with the chronics this time

This is to test the environment can be used by grid2op. This is what the "case118_l2rpn" will look like for the competitions.

**NB** The "Balthazar code" is fully compatible with the "GridStateFromFileWithForecasts". So it is useful to use this class to load back the data. If the data generation process does not provide the same utilities, it is not a problem to write another class, like "GridStateFromFileWithForecasts" that can read its format.

### Correct the bug in element 7_4_173

In [ ]:
# NO YOU SHOULD NOT DO THAT BUT BE ABLE TO RUN ALL SCENARIOS IN THE RUNNER.

# Chose generated scenario
# scenario = 'Scenario_15'
# scenario_path = os.path.join(OUTPUT_FOLDER, str(year), scenario)


In [ ]:
generation_output_folder #='{output_folder}/generation/{CASE}/{start_date}'

In [ ]:
from grid2op.Chronics import Multifolder, GridStateFromFileWithForecasts
from grid2op.Parameters import Parameters
try:
    from lightsim2grid.LightSimBackend import LightSimBackend
    backend = LightSimBackend()
except:
    from grid2op.Backend import PandaPowerBackend
    backend = PandaPowerBackend()
    print("You might need to install the LightSimBackend (provisory name) to gain massive speed up")
# don't disconnect powerline on overflow, the thermal limit are not set for now, it would not make sens
param = Parameters()
param.init_from_dict({"NO_OVERFLOW_DISCONNECTION": True})

env = grid2op.make("blank",  # to generate a blank environment
                   grid_path=grid_path, # assign it the 118 grid
                   chronics_class=Multifolder, # tell it to change nothing (not the most usable environment...)
                   data_feeding_kwargs= {
                       "path": os.path.abspath(generation_output_folder), "gridvalueClass": GridStateFromFileWithForecasts},
                   param=param,
                   backend=backend
                  )
# If you remove the "GridStateFromFileWithForecasts", from above, chronics will NOT be loaded properly.
# GridStateFromFileWithForecasts is the format used for the competition, so it is mandatory that this works!
# WITHOUT ANY MODIFICATIONS

# Beside the environment should be able to load all data generated, and not one episode.
# so please look in grid2op for compatible formats. This is not a valid format.

In [ ]:
print("Have all the chronics been loaded: {}".format(len(env.chronics_handler.real_data.subpaths) == n_scenarios))

In [ ]:
#set env thermal limit to 1 by default
th_lim = np.ones(env.n_line, dtype=np.float)
env.set_thermal_limit(th_lim)

And now we can test that we can use a Runner, store the results, and plot the flows on the powerline for example

# IV Validate the generation process

For that we use a runner, that will compute the powerflows with a "do nothing" agent, and we prevent it to disconnect any power line, even if they are on overflow.

In [ ]:
generation_output_folder

In [ ]:
from grid2op.Runner import Runner
import tempfile
from tqdm.notebook import tqdm

path_data_saved = os.path.join(os.path.abspath(os.path.join(generation_output_folder, os.pardir)), 'agent_results')#, scenario_name)
os.makedirs(path_data_saved, exist_ok=True)

nb_episode = 1#10
NB_CORE = 1#4
# nb_episode = n_scenarios
#nb_steps = 400
runner = Runner(**env.get_params_for_runner())



In [ ]:
# here you might need to change "nb_episode" if you generated more than one scenario
# this might really take some times... 4 mins per scenario per week [on one core]
res = runner.run(nb_episode=nb_episode,nb_process=NB_CORE, path_save=path_data_saved, pbar=tqdm)

In [ ]:
!ls $path_data_saved

In [ ]:
path_data_saved

Then we can study the results, for example by loading the chronics, extracting prod p, load p etc.

In [ ]:
from grid2op.EpisodeData import EpisodeData
import numpy as np
from tqdm.notebook import tqdm

data_this_episode = EpisodeData.from_disk(path_data_saved, 'Scenario_0')

In [ ]:
path_data_saved

In [ ]:
flows_a = pd.DataFrame(np.array([obs.a_or for obs in data_this_episode.observations]))
loads_p = pd.DataFrame(np.array([obs.load_p for obs in data_this_episode.observations]))
prods_p = pd.DataFrame(np.array([obs.prod_p for obs in data_this_episode.observations]))


### Check losses & prods

In [ ]:
ProdTotal=prods_p.sum(axis=1)
ConsoTotal=loads_p.sum(axis=1)
Pertes=(ProdTotal-ConsoTotal)/ConsoTotal

tauxDePerte=Pertes.mean()
Pertes.describe()

In [ ]:

TotalLossesRatio=pd.DataFrame(np.array([(np.sum(obs.prod_p)-np.sum(obs.load_p))/np.sum(obs.load_p) for obs in data_this_episode.observations]))


print('average loss rate is: '+ str(TotalLossesRatio.mean()))
TotalLossesRatio.iplot(kind='scatter', filename='cufflinks/cf-simple-line')

Productions

In [ ]:
prods_p.iplot(kind='scatter', filename='cufflinks/cf-simple-line')

In [ ]:
nuclear_idx = [i for i in range(len(env.gen_type)) if env.gen_type[i] == 'nuclear'] 
hydro_idx = [i for i in range(len(env.gen_type)) if env.gen_type[i] == 'hydro']
thermal_idx = [i for i in range(len(env.gen_type)) if env.gen_type[i] == 'thermal']

In [ ]:

colnames_p=prods_p.columns.values
prods_p_perType=pd.DataFrame()
prods_p_perType['nuclear']=prods_p[colnames_p[nuclear_idx]].sum(axis=1)
prods_p_perType['hydro']=prods_p[colnames_p[hydro_idx]].sum(axis=1)
prods_p_perType['thermal']=prods_p[colnames_p[thermal_idx]].sum(axis=1)
    

In [ ]:
prods_p_perType[['nuclear','hydro','thermal']].iplot(kind='scatter', filename='cufflinks/cf-simple-line')

### R3 zone load and production balance

In [ ]:
net = env118_withoutchron.backend._grid

load_p_with_names = loads_p.copy()
load_p_with_names.columns = data_this_episode.load_names
# Sort same order as net
load_p_with_names = load_p_with_names[net.load.name]

prods_p_with_names = prods_p.copy()
prods_p_with_names.columns = data_this_episode.prod_names
# Sort same order as net
prods_p_with_names = prods_p_with_names[net.gen.name]

In [ ]:
region = 'R3'

prods_names = net.gen[net.gen.zone.isin([region])].name.tolist()
loads_names = net.load[net.load.zone.isin([region])].name.tolist()

agg_vals = pd.concat([prods_p_with_names[prods_names].sum(axis=1).to_frame(f'agg_prods_in_{region}'), 
                      load_p_with_names[loads_names].sum(axis=1).to_frame(f'agg_loads_in_{region}'),
                      ], axis=1)

agg_vals.iplot()

In [ ]:
flows_mw = pd.DataFrame(np.array([obs.p_or for obs in data_this_episode.observations]))
flows_mw.iplot()

In [ ]:
indexHighFlow=np.abs(flows_mw).max()[np.abs(flows_mw).max()>200].index
np.abs(flows_mw).max()[np.abs(flows_mw).max()>200]

In [ ]:
np.abs(flows_mw).max()[6]

In [ ]:
print(env.line_or_to_subid[indexHighFlow])
print(env.line_ex_to_subid[indexHighFlow])

Go back to **II)** if results are not satisfying.